In [9]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags
from scipy.sparse import diags
from scipy.fftpack import fft2, ifft2
from scipy.integrate import solve_ivp

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags
from scipy.integrate import solve_ivp
Lx = 20
nx = 64
N = 64**2
delta =  Lx / nx
e0 = np.zeros((N, 1))  
e1 = np.ones((N, 1))  
e2 = np.copy(e1)   
e4 = np.copy(e0) 

for j in range(1, nx+1):
    e2[nx*j-1] = 0  
    e4[nx*j-1] = 1  


e3 = np.zeros_like(e2)
e3[1:N] = e2[0:N-1]
e3[0] = e2[N-1]

e5 = np.zeros_like(e4)
e5[1:N] = e4[0:N-1]
e5[0] = e4[N-1]


diagonals = [e1.flatten(), e1.flatten(), e5.flatten(), 
             e2.flatten(), -4 * e1.flatten(), e3.flatten(), 
             e4.flatten(), e1.flatten(), e1.flatten()]
offsets = [-(N-nx), -nx, -nx+1, -1, 0, 1, nx-1, nx, (N-nx)]

matA = spdiags(diagonals, offsets, N, N).toarray()
A = matA / ((delta) ** 2)
# A[0, 0] = 2
# print(A)
e1 = np.ones((N, 1))  

diagonals = [e1.flatten(), -1 * e1.flatten(), e1.flatten(), -1 * e1.flatten()]
offsets = [nx, -nx, -N+nx, N-nx]

matB = spdiags(diagonals, offsets, N, N).toarray()
B = matB / (2 * delta)
# print(B)
diagonals_C = [e1.flatten(), -e1.flatten()]
offsets_C = [1, -1]

matC = spdiags(diagonals_C, offsets_C, N, N).toarray()
# print(C)


for i in range(nx):
    matC[i * nx, i * nx - 1] = 0
    matC[i * nx - 1, i * nx] = 0
    matC[i * nx, (i + 1) * nx - 1] = -1
    matC[(i + 1) * nx - 1, i * nx] = 1

C = matC / (2 * delta)

In [12]:
def spc_rhs(t, w0, nx, ny, K, nu):
    w2d = w0.reshape((nx, ny))
    wt = fft2(w2d)
    psit = -wt / K
    psi = np.real(ifft2(psit))
    psi = psi.flatten()
    rhs = (nu * A.dot(w0) + B.dot(w0) * C.dot(psi) - C.dot(w0) * B.dot(psi))
    return rhs

tspan = np.arange(0, 4+0.5, 0.5)
nu = 0.001
Lx, Ly = 20, 20
nx, ny = 64, 64
N = nx * ny

# Define spatial domain and initial conditions
x2 = np.linspace(-Lx/2, Lx/2, nx + 1)
x = x2[:nx]
y2 = np.linspace(-Ly/2, Ly/2, ny + 1)
y = y2[:ny]
X, Y = np.meshgrid(x, y)
w = np.exp(-X**2 - Y**2 / 20)

# Define spectral k values
kx = (2 * np.pi / Lx) * np.concatenate((np.arange(0, nx/2), np.arange(-nx/2, 0)))
kx[0] = 1e-6
ky = (2 * np.pi / Ly) * np.concatenate((np.arange(0, ny/2), np.arange(-ny/2, 0)))
ky[0] = 1e-6
KX, KY = np.meshgrid(kx, ky)
K = KX**2 + KY**2

w0 = w.flatten()
wtsol = solve_ivp(spc_rhs, [0, 4], w0, method= "RK45", t_eval=tspan, args=(nx, ny, K, nu))
wtsol = wtsol.y
print(wtsol)

[[2.50656748e-46 3.54964961e-45 1.85768096e-44 ... 1.85272739e-42
  4.67439366e-42 1.11058944e-41]
 [1.17762859e-43 6.53319004e-43 2.62245678e-42 ... 1.58946594e-40
  3.56498221e-40 7.46793534e-40]
 [4.55107657e-41 1.92934361e-40 6.43507373e-40 ... 2.49607447e-38
  5.08478333e-38 9.69634778e-38]
 ...
 [1.96785570e-38 1.23357409e-37 5.47591944e-37 ... 4.35250271e-35
  1.02347073e-34 2.23043626e-34]
 [6.19028421e-41 5.33777602e-40 2.93883953e-39 ... 3.67992748e-37
  9.45792605e-37 2.23822117e-36]
 [1.60178709e-43 1.99380943e-42 1.41671738e-41 ... 2.79962856e-39
  7.95937714e-39 2.07748843e-38]]
